<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Функции-и-библиотеки" data-toc-modified-id="Функции-и-библиотеки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Функции и библиотеки</a></span></li><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#подготовка-признаков" data-toc-modified-id="подготовка-признаков-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>подготовка признаков</a></span></li><li><span><a href="#BERT-(подготовка-признаков)" data-toc-modified-id="BERT-(подготовка-признаков)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>BERT (подготовка признаков)</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li></ul></li><li><span><a href="#Финальное-тестирование" data-toc-modified-id="Финальное-тестирование-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Финальное тестирование</a></span></li><li><span><a href="#Заключение" data-toc-modified-id="Заключение-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Заключение</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. 

Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.


Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.


Постройте модель со значением метрики качества F1 не меньше 0.75.

## Функции и библиотеки

In [62]:
import pandas as pd

import numpy as np

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score

from sklearn.utils import shuffle

from sklearn.ensemble import RandomForestClassifier

from tqdm import notebook


import re 


import torch
import transformers


import warnings
warnings.filterwarnings('ignore')

In [63]:
!pip install pymystem3 -U

In [64]:
!pip install torch -U

In [65]:
!pip install transformers -U

In [66]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    text = text.split()
    return " ".join(text)

In [67]:
def downsample(features, target):
      
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    fraction = round(len(target_ones)/len(target_zeros),2)

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

## Обзор данных

In [68]:
try:
    df = pd.read_csv ('D:/Data for projects/toxic_comments.csv')


except: 
    df = pd.read_csv ('/datasets/toxic_comments.csv')


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [70]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


## Предобработка данных

In [71]:
df['text'] = df['text'].apply(clear_text)

## Обучение моделей

### подготовка признаков

вычислительных ресурсов моего компьютера не хватает для обработки всего датасета, по этому методом эксперимента мной определен мксимальный размер выборки на которой и будет проводится обучение моделей 

так же проведем балансировку целевых признаков в тренировочной выборке

In [72]:
state = np.random.RandomState(12345)
batch = df.sample(frac=0.05, replace=False, random_state=state)

In [73]:
batch['text'], batch['toxic'] = downsample(batch['text'], batch['toxic'])

In [74]:
batch['toxic'].value_counts()

1.0    797
0.0    790
Name: toxic, dtype: int64

In [75]:
batch = batch.dropna()

In [76]:
batch = batch.reset_index(drop=True)

сформируем тренировочную и тестовую выборки

In [77]:
batch, batch_test = train_test_split(batch, test_size=0.25, random_state=404)

In [78]:
batch = batch.reset_index(drop=True)
batch.shape

(1190, 2)

In [79]:
batch_test = batch_test.reset_index(drop=True)
batch_test.shape

(397, 2)

In [80]:
batch['toxic'].value_counts()

1.0    598
0.0    592
Name: toxic, dtype: int64

из не задействованных данных соберу батч для тестирования модели

batch_test = df.sample(frac=0.002, replace=False, random_state=state).reset_index(drop=True)

### BERT (подготовка признаков)

In [81]:
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

In [82]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [83]:
tokenized = batch['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (821 > 512). Running this sequence through the model will result in indexing errors


In [84]:
tokenized_test = batch_test['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [85]:
tokenized

0       [101, 2123, 2102, 2568, 2023, 5195, 21025, 210...
1       [101, 2115, 28297, 1999, 2437, 3167, 8767, 200...
2       [101, 1042, 1057, 1039, 1047, 1045, 1050, 1043...
3       [101, 2175, 6616, 4426, 10931, 2304, 22673, 51...
4       [101, 4364, 2066, 2017, 5754, 6977, 2033, 2061...
                              ...                        
1185    [101, 2023, 2003, 9951, 4998, 2013, 1996, 4431...
1186    [101, 7592, 9587, 3406, 7592, 4875, 3654, 2100...
1187    [101, 8185, 2964, 3718, 8185, 2964, 2013, 2023...
1188    [101, 2500, 3188, 2000, 2008, 2051, 2323, 2022...
1189    [101, 1040, 15922, 2006, 2251, 2106, 2017, 211...
Name: text, Length: 1190, dtype: object

In [86]:
tokenized_test

0      [101, 4433, 2316, 4681, 1045, 2031, 2787, 2000...
1      [101, 2008, 1055, 5875, 2138, 1045, 2310, 4384...
2      [101, 2023, 2079, 5051, 2003, 2108, 3825, 2000...
3      [101, 1996, 2208, 2003, 2746, 2041, 2399, 1439...
4      [101, 2251, 11396, 4067, 2017, 2097, 2228, 205...
                             ...                        
392    [101, 20948, 2065, 1057, 2123, 1056, 2113, 205...
393    [101, 2026, 18006, 25380, 1045, 2001, 1999, 10...
394    [101, 4638, 6300, 20255, 11721, 8524, 2099, 10...
395    [101, 1037, 16983, 17906, 1055, 3193, 2339, 20...
396    [101, 4283, 1045, 2064, 2156, 2009, 2003, 2006...
Name: text, Length: 397, dtype: object

In [87]:
counter_list = []
counter = 0
for _ in tokenized:
    if np.array(_).size >511:
        counter_list.append(counter)
    counter +=1
    
#counter_list    

In [88]:
counter_list_test = []
counter = 0
for _ in tokenized_test:
    if np.array(_).size >511:
        counter_list_test.append(counter)
    counter +=1
    
#counter_list_test   

In [89]:
tokenized = tokenized.drop(counter_list)

In [90]:
tokenized_test = tokenized_test.drop(counter_list_test)

In [91]:
n = tokenized.apply(lambda x: np.array(x).size).max()
n

495

In [92]:
n = tokenized_test.apply(lambda x: np.array(x).size).max()
n

494

In [93]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [94]:
max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

In [95]:
np.array(padded).shape

(1172, 495)

In [96]:
np.array(padded_test).shape

(392, 494)

In [97]:
padded= padded.astype('int64')

In [98]:
padded_test = padded_test.astype('int64')

In [99]:
attention_mask = np.where(padded != 0, 1, 0).astype('int64')
attention_mask.shape

(1172, 495)

In [100]:
attention_mask_test = np.where(padded_test != 0, 1, 0).astype('int64')
attention_mask_test.shape

(392, 494)

In [101]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [102]:
input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

In [103]:
labels = batch['toxic'].drop(counter_list)

In [104]:
target_test = batch_test['toxic'].drop(counter_list_test)

In [105]:
%%time

batch_size = 50
embeddings = []
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings)        

  0%|          | 0/23 [00:00<?, ?it/s]

Wall time: 8h 36min 4s


In [106]:
%%time


batch_size = 50
embeddings = []
for i in notebook.tqdm(range(input_ids_test.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids_test[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask_test[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features_test = np.concatenate(embeddings)        

  0%|          | 0/7 [00:00<?, ?it/s]

Wall time: 12min 43s


input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()

### LogisticRegression

In [123]:
labels = labels[:1150]
labels.shape

(1150,)

In [125]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [126]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [127]:
lr_clf.score(test_features, test_labels)

0.8368055555555556

In [128]:
f1_score(np.array(test_labels), lr_clf.predict(test_features))

0.8448844884488449

### RandomForestClassifier

In [129]:
model_RFC = RandomForestClassifier(random_state=404, n_estimators=34) 

model_RFC.fit(train_features, train_labels) # обучение модели

f1_score(np.array(test_labels), model_RFC.predict(test_features))

0.785234899328859

## Финальное тестирование

In [133]:
target_test = target_test[:350]

In [130]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.472 (+/- 0.09)


In [135]:
f1_score(np.array(target_test), lr_clf.predict(features_test))

0.8352941176470587

In [136]:
f1_score(np.array(target_test), model_RFC.predict(features_test))

0.778115501519757

## Заключение

в рамках настоящего ислледования проведено обучение моделей (логистической регрессии и случайного леса) с использованием предобученной модели BERT для определения токсичных комментариев

результаты метрики f1 для обеих моделей соответствуют требуемым - не меньше 0.75

